In [1]:
import sys
import time

sys.path.append("..")
import pandas as pd
import solv_fzcp as sfzcp
import uvarprob as uvpr

In [2]:
test_file = "../test_casado2.csv"


In [3]:
def read_problems(fname):
    data = pd.read_csv(fname, index_col='name', comment='#')
    return data


df = read_problems(test_file)
df

,objective,a,b,min_f,min_x
name,,,,,
1,-0.5*x^2*(log(x))+5,0.2,7.0,0,0
3,-(sqrt(x))*sin(x)+1,0.2,7.0,0,0
4,x*sin(x)+sin(10.0*x/3.0)+log(x)-0.84*x+1.3,0.2,7.0,0,0
5,x+sin(5.0*x),0.2,7.0,0,0
7,sin(x)*cos(x)-1.5*(sin(x))^2+1.2,0.2,7.0,0,0
9,2.0*sin(x)*exp(-x),0.2,7.0,0,0
10,(3*x-1.4)*sin(18.0*x)+1.7,0.2,7.0,0,0
11,(x+1)^3/x^2-7.1,0.2,7.0,0,0
13,12+cos(2.0*x+1)+2*cos(3.0*x+2)+3*cos(4.0*x+3)...,0.2,7.0,0,0


In [4]:
points_db = {}
psl_lipint_points_list = []
psl_lip_points_list = []
psqe_lipint_points_list = []
psqe_lip_points_list = []
bnb2_lipint_points_list = []
bnb2_lip_points_list = []

In [5]:
def log_point(x, points_list):
    points_list.append(x)

In [6]:
import ia_math_fun as iaf
import interval_arithmetics as ia

for test in df.itertuples():
    print(test.Index)
    points_db[test.Index] = {'psl_lipint_points_list': [], 'psl_lip_points_list': [], 'psqe_lipint_points_list': [],
                             'psqe_lip_points_list': [], 'bnb2_pslint_points_list': [], 'bnb2_psl_points_list': [],
                             'bnb2_lipint_points_list': [],
                             'bnb2_lip_points_list': []}

    # #
    # prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x,
    #                           lambda x: log_point(x, points_db[test.Index]['bnb2_pslint_points_list']))
    # period = 0
    # T1 = time.perf_counter()
    # print('Piyavskii, Lipschitz interval:',
    #       sfzcp.correctly(prob, symm=False, epsilon=1e-10, global_lipschitz_interval=False, estimator=1, reduction=1,
    #                        period_comp_lip=period))
    # T2 = time.perf_counter()
    # print('time=', (T2 - T1)*1000, 'period:', period)
    #
    iaf.set_log_taylor_terms_number(19)
    iaf.set_cos_taylor_terms_number(19)
    iaf.set_sin_taylor_terms_number(19)
    iaf.set_exp_taylor_terms_number(19)
    ia.set_precision(19)
    prob = uvpr.UniVarProblem(name=test.Index, objective=test.objective, a=test.a, b=test.b, min_f=test.min_f,
                              min_x=test.min_x,
                              logger=lambda x: log_point(x, points_db[test.Index]['bnb2_pslint_points_list']),
                              preproc=True, correctly=True)
    period = 0
    T1 = time.perf_counter()
    print('PSQE:',
          sfzcp.correctly(prob, symm=False, epsilon=1e-9, global_lipschitz_interval=False, estimator=2, reduction=1, adaptive=False))
    T2 = time.perf_counter()
    print('time=', (T2 - T1) * 1000, 'period:', period)
    T1 = time.perf_counter()
    print('Linear:',
          sfzcp.correctly(prob, symm=False, epsilon=1e-9, global_lipschitz_interval=False, estimator=1, reduction=1, adaptive=False))
    T2 = time.perf_counter()
    print('time=', (T2 - T1) * 1000, 'period:', period)

1
PSQE: TestResult(nsteps=4, first_crossing_zero_point=[[3.011690771046723688, 3.011690771046755090]])
time= 16.033916996093467 period: 0
Linear: TestResult(nsteps=6, first_crossing_zero_point=[[3.011690771046688014, 3.011690771046755147]])
time= 12.022374998196028 period: 0
3
PSQE: TestResult(nsteps=6, first_crossing_zero_point=[[1.174796171291449297, 1.174796171291492291]])
time= 77.53070798935369 period: 0
Linear: TestResult(nsteps=9, first_crossing_zero_point=[[1.174796171291433028, 1.174796171291503439]])
time= 58.01904200052377 period: 0
4
PSQE: TestResult(nsteps=6, first_crossing_zero_point=[[2.960913794441498109, 2.960913794442274295]])
time= 146.28416699997615 period: 0
Linear: TestResult(nsteps=10, first_crossing_zero_point=[[2.960913794425994462, 2.960913794465647099]])
time= 134.3391250120476 period: 0
5
PSQE: TestResult(nsteps=8, first_crossing_zero_point=[[0.8209239701115810880, 0.8209239701115811599]])
time= 43.33279198908713 period: 0
Linear: TestResult(nsteps=9, first_

In [13]:
import ia_math_fun as iaf
import interval_arithmetics as ia

for test in df.itertuples():
    print(test.Index)
    points_db[test.Index] = {'psl_lipint_points_list': [], 'psl_lip_points_list': [], 'psqe_lipint_points_list': [],
                             'psqe_lip_points_list': [], 'bnb2_pslint_points_list': [], 'bnb2_psl_points_list': [],
                             'bnb2_lipint_points_list': [],
                             'bnb2_lip_points_list': []}

    iaf.set_log_taylor_terms_number(11)
    iaf.set_cos_taylor_terms_number(11)
    iaf.set_sin_taylor_terms_number(11)
    iaf.set_exp_taylor_terms_number(11)
    ia.set_precision(11)
    prob = uvpr.UniVarProblem(name=test.Index, objective=test.objective, a=test.a, b=test.b, min_f=test.min_f,
                              min_x=test.min_x,
                              logger=lambda x: log_point(x, points_db[test.Index]['bnb2_pslint_points_list']),
                              preproc=True, correctly=True)

    T1 = time.perf_counter()
    print('PSQE:',
          sfzcp.correctly(prob, symm=False, epsilon=1e-7, global_lipschitz_interval=False, estimator=2, reduction=1,
                          adaptive=False))
    T2 = time.perf_counter()
    print('time=', (T2 - T1) * 1000, 'period:', period)

    T1 = time.perf_counter()
    print('Linear:',
          sfzcp.correctly(prob, symm=False, epsilon=1e-7, global_lipschitz_interval=False, estimator=1, reduction=1,
                          adaptive=False))
    T2 = time.perf_counter()
    print('time=', (T2 - T1) * 1000, 'period:', period)

1
PSQE: TestResult(nsteps=4, first_crossing_zero_point=[[3.0116907674, 3.0116907747]])
time= 13.70570799917914 period: 0
PSQE: TestResult(nsteps=4, first_crossing_zero_point=[[3.0116907707, 3.0116907744]])
time= 17.08154199877754 period: 0
Linear: TestResult(nsteps=6, first_crossing_zero_point=[[3.0116907707, 3.0116907744]])
time= 11.441750000813045 period: 0
3
PSQE: TestResult(nsteps=9, first_crossing_zero_point=[[1.1747961711, 1.1747961714]])
time= 57.50412499764934 period: 0
PSQE: TestResult(nsteps=6, first_crossing_zero_point=[[1.1747961700, 1.1747961745]])
time= 37.25716700137127 period: 0
Linear: TestResult(nsteps=9, first_crossing_zero_point=[[1.1747961711, 1.1747961714]])
time= 27.772832996561192 period: 0
4
PSQE: TestResult(nsteps=8, first_crossing_zero_point=[[2.9609137934, 2.9609137958]])
time= 83.18216599582229 period: 0
PSQE: TestResult(nsteps=6, first_crossing_zero_point=[[2.9609137892, 2.9609138027]])
time= 77.25291600218043 period: 0
Linear: TestResult(nsteps=10, first_

In [14]:
import ia_math_fun as iaf
import interval_arithmetics as ia

for test in df.itertuples():
    print(test.Index)
    points_db[test.Index] = {'psl_lipint_points_list': [], 'psl_lip_points_list': [], 'psqe_lipint_points_list': [],
                             'psqe_lip_points_list': [], 'bnb2_pslint_points_list': [], 'bnb2_psl_points_list': [],
                             'bnb2_lipint_points_list': [],
                             'bnb2_lip_points_list': []}

    iaf.set_log_taylor_terms_number(11)
    iaf.set_cos_taylor_terms_number(11)
    iaf.set_sin_taylor_terms_number(11)
    iaf.set_exp_taylor_terms_number(11)
    ia.set_precision(11)
    prob = uvpr.UniVarProblem(name=test.Index, objective=test.objective, a=test.a, b=test.b, min_f=test.min_f,
                              min_x=test.min_x,
                              logger=lambda x: log_point(x, points_db[test.Index]['bnb2_pslint_points_list']),
                              preproc=True, correctly=True)
    period = 0
    T1 = time.perf_counter()
    steps = sfzcp.correctly(prob, symm=False, epsilon=1e-6, global_lipschitz_interval=True, estimator=2, reduction=1,
                            adaptive=False).nsteps
    T2 = time.perf_counter()
    print('Quadratic, global lipschitz')
    print('steps=', steps, 'time=', (T2 - T1) * 1000)
    T1 = time.perf_counter()
    steps = sfzcp.correctly(prob, symm=False, epsilon=1e-6, global_lipschitz_interval=True, estimator=1, reduction=1,
                            adaptive=False).nsteps
    T2 = time.perf_counter()
    print('Linear, global lipschitz')
    print('steps=', steps, 'time=', (T2 - T1) * 1000)

    T1 = time.perf_counter()
    steps = sfzcp.correctly(prob, symm=False, epsilon=1e-6, global_lipschitz_interval=False, estimator=2, reduction=1,
                            adaptive=False).nsteps
    T2 = time.perf_counter()
    print('Quadratic, local lipschitz')
    print('steps=', steps, 'time=', (T2 - T1) * 1000)
    T1 = time.perf_counter()
    steps = sfzcp.correctly(prob, symm=False, epsilon=1e-6, global_lipschitz_interval=False, estimator=1, reduction=1,
                            adaptive=False).nsteps
    T2 = time.perf_counter()
    print('Linear, local lipschitz')
    print('steps=', steps, 'time=', (T2 - T1) * 1000)

    T1 = time.perf_counter()
    steps = sfzcp.correctly(prob, symm=False, epsilon=1e-6, global_lipschitz_interval=False, estimator=2, reduction=1,
                            adaptive=True).nsteps
    T2 = time.perf_counter()
    print('Quadratic, adaptive lipschitz v1')
    print('steps=', steps, 'time=', (T2 - T1) * 1000)


1
Quadratic, global lipschitz
steps= 4 time= 10.997584002325311
Linear, global lipschitz
steps= 40 time= 34.88479199586436
Quadratic, local lipschitz
steps= 4 time= 9.52766599948518
Linear, local lipschitz
steps= 6 time= 8.520167000824586
Quadratic, adaptive lipschitz v1
steps= 4 time= 9.515291996649466
3
Quadratic, global lipschitz
steps= 9 time= 51.78899999009445
Linear, global lipschitz
steps= 45 time= 78.01404199562967
Quadratic, local lipschitz
steps= 6 time= 44.758291987818666
Linear, local lipschitz
steps= 8 time= 26.621667013387196
Quadratic, adaptive lipschitz v1
steps= 9 time= 61.36333299218677
4
Quadratic, global lipschitz
steps= 8 time= 86.10983300604858
Linear, global lipschitz
steps= 35 time= 160.1434999902267
Quadratic, local lipschitz
steps= 6 time= 72.47558300150558
Linear, local lipschitz
steps= 10 time= 63.57320801180322
Quadratic, adaptive lipschitz v1
steps= 8 time= 81.2344169971766
5
Quadratic, global lipschitz
steps= 8 time= 18.014457993558608
Linear, global lips